In [682]:

import logging
import sys

logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

# Add a StreamHandler
handler = logging.StreamHandler(sys.stdout)
handler.setLevel(logging.INFO)
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
handler.setFormatter(formatter)
logger.addHandler(handler)

import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from scipy.optimize import minimize
import warnings
from sklearn.exceptions import DataConversionWarning
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error, mean_absolute_percentage_error
from datetime import datetime


warnings.filterwarnings(action='ignore', category=DataConversionWarning)

from scipy.optimize import minimize

from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error

from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import FunctionTransformer
from sklearn.pipeline import Pipeline

from sklearn.covariance import LedoitWolf



In [683]:
print(f"__name__ value: {__name__}")
logger = logging.getLogger("my_main_script")

__name__ value: __main__


In [684]:

# 模型参数
rf_params = {
    'n_estimators': 1000,
    'max_depth': 10,
    'min_samples_split': 5,
    'min_samples_leaf': 2,
    'max_features': 'sqrt',
    'random_state': 42,
    'warm_start' : True
}

# 训练模型
rf_model = RandomForestRegressor(**rf_params)

In [685]:
rf_data = pd.read_csv('model_data.csv')
rf_data['date'] = pd.to_datetime(rf_data['date'])
rf_data.set_index(['date'], inplace=True)

# 确定训练集的开始和结束日期
train_start = rf_data.index.min()
train_end = train_start + pd.DateOffset(months=6)
test_start = train_end
test_end = test_start + pd.DateOffset(months=1)

# 分割数据集为训练集（前6个月）和预测集（第7个月）
rf_data_for_training = rf_data[(rf_data.index >= train_start) & (rf_data.index < train_end)]
rf_data_using_model_predicting = rf_data[(rf_data.index >= test_start) & (rf_data.index < test_end)]

# 定义特征和目标变量
features = rf_data.columns[~rf_data.columns.isin(['TICKER', 'RET'])].tolist()
X_train = rf_data_for_training[features]
y_train = rf_data_for_training['RET']
X_test = rf_data_using_model_predicting[features]
y_test = rf_data_using_model_predicting['RET']

print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)
print("Training period:", train_start, "to", train_end)
print("Testing period:", test_start, "to", test_end)
print(X_train.shape,y_train.shape,X_test.shape,y_test.shape)




(157, 39) (157,) (26, 39) (26,)
Training period: 2014-07-31 00:00:00 to 2015-01-31 00:00:00
Testing period: 2015-01-31 00:00:00 to 2015-02-28 00:00:00
(157, 39) (157,) (26, 39) (26,)


In [686]:
from sklearn.pipeline import Pipeline
import pandas as pd
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import FunctionTransformer
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import Pipeline
# 训练模型

# 模型参数
rf_params = {
    'n_estimators': 100,
    # 'max_depth': 21,
    # 'min_samples_split': 2,
    # 'min_samples_leaf': 4,
    # 'max_features': 'auto',
    'random_state': 42
}

# 2. 模型创建和训练
rf_model = RandomForestRegressor(**rf_params)
pipeline = Pipeline([('rf', rf_model)])

pipeline.fit(X_train.values, y_train)

# 3. 模型评估
y_pred = pipeline.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
mape = mean_absolute_percentage_error(y_test, y_pred)

r2 = r2_score(y_test, y_pred)

print(f"Mean Squared Error: {mse}")
print(f"Mean Absolute Error: {mape}")

print(f"R-squared Score: {r2}")

Mean Squared Error: 0.001071484773941469
Mean Absolute Error: 1.3535129710986291
R-squared Score: -0.0669844862474871


/opt/homebrew/anaconda3/lib/python3.10/site-packages/sklearn/base.py:413: UserWarning: X has feature names, but RandomForestRegressor was fitted without feature names
  warnings.warn(


In [687]:
X_predict = rf_data_using_model_predicting[features]
y_actual = rf_data_using_model_predicting['RET']

# 在最后一个月的预测集上进行预测
y_predict = pipeline.predict(X_predict)

r2_predict = r2_score(y_actual, y_predict)
mse_predict = mean_squared_error(y_actual, y_predict)
mape_predict = mean_absolute_percentage_error(y_actual, y_predict)

print(f'Prediction R²: {r2_predict:.4f}')
print(f'Prediction MSE: {mse_predict:.4f}')
print(f'Prediction MAPE: {mape_predict:.4f}')

Prediction R²: -0.0670
Prediction MSE: 0.0011
Prediction MAPE: 1.3535


/opt/homebrew/anaconda3/lib/python3.10/site-packages/sklearn/base.py:413: UserWarning: X has feature names, but RandomForestRegressor was fitted without feature names
  warnings.warn(


In [688]:
variables_to_keep = ['Size', 'bm', 'Momentum', 'r2_1', 'r12_2', 'r12_7', 'ShortTermReversal',
       'market_cap', 'DollarVolume', 'spread', 'RET', 'idio_vol', 'Beta',
       'BetaSquared', 'pe_op_basic', 'pe_op_dil', 'Volatility', 'SUV',
       '52_week_high', 'Rel_to_high', 'TB3MS', 'T10Y2Y', 'BAA10Y', 'roa',
       'roe', 'roce', 'debt_assets', 'debt_capital', 'de_ratio', 'cash_ratio',
       'quick_ratio', 'curr_ratio', 'at_turn', 'inv_turn', 'accrual', 'vwretd',
       'ewretd', 'sprtrn', 'LME', 'LTurnover', 'TICKER']

In [689]:
opt_data = pd.read_csv('model_data.csv')
opt_data['date'] = pd.to_datetime(opt_data['date'])
opt_data.set_index(['date', 'TICKER'], inplace=True)
opt_data.sort_index(inplace=True)

In [690]:
opt_features = opt_data.columns[~opt_data.columns.isin(['RET','TICKER'])].tolist()
target = ['RET']

In [691]:
featured_data = pd.read_csv('final_data_after_missing_value_handle_with_better_features.csv')
featured_data['date'] = pd.to_datetime(featured_data['date'])
featured_data.set_index(['date', 'TICKER'], inplace=True)
featured_data.sort_index(inplace=True)


In [692]:
def display_portfolio_composition(weights, stocks):
    """
    Display the composition of the portfolio with ticker names and weights.
    
    Parameters:
    weights (numpy.array): Optimized weights for the portfolio
    stocks (pandas.Index or list): The stock tickers in the portfolio
    threshold (float): Minimum weight to include in the display (default: 1%)
    
    Returns:
    pandas.DataFrame: A dataframe showing the portfolio composition
    str: A string representation of the portfolio composition
    """
    # Create a dataframe with tickers and weights
    portfolio = pd.DataFrame({
        'Ticker': stocks,
        'Weight': weights
    })
    threshold=0.01
    # Sort by weight in descending order
    portfolio = portfolio.sort_values('Weight', ascending=False)
    
    # Filter out weights below the threshold
    portfolio = portfolio[portfolio['Weight'] >= threshold]
    
    # Calculate the sum of displayed weights
    displayed_weight_sum = portfolio['Weight'].sum()
    
    # Add a row for "Others" if necessary
    if displayed_weight_sum < 1:
        others_weight = 1 - displayed_weight_sum
        others_row = pd.DataFrame({
            'Ticker': ['Others'],
            'Weight': [others_weight]
        })
        portfolio = pd.concat([portfolio, others_row])
    
    # Format weights as percentages
    portfolio['Weight'] = portfolio['Weight'].apply(lambda x: f"{x:.2%}")
    
    # Reset index for clean display
    portfolio = portfolio.reset_index(drop=True)
    
    # Create a string representation
    portfolio_str = portfolio.to_string(index=False)
    
    return portfolio, portfolio_str


In [712]:
from sklearn.base import clone
from sklearn.utils import resample


def get_data_for_date_range(df, start_date, end_date):
    # return df.loc[start_date:end_date]
    if start_date is None:
        data = df.loc[:end_date]
    else:
        data = df.loc[start_date:end_date]
    if data.empty:
        logger.warning(f"No data found between {start_date} and {end_date}")
    return data

def prepare_training_data(df, features, target):
    X = df[features]
    y = df[target]
    return X, y

def fit_and_predict(model, X_train, y_train, X_test):
    # logger.info(f"fit_and_predict: X_train shape: {X_train.shape}, type: {type(X_train)}")
    # logger.info(f"fit_and_predict: y_train shape: {y_train.shape}, type: {type(y_train)}")
    # logger.info(f"fit_and_predict: X_test shape: {X_test.shape}, type: {type(X_test)}")
    
    # Train the new model
    # logger.info("start fitting")
    model.fit(X_train.values, y_train)
    # logger.info("end fitting")

    predictions = model.predict(X_test.values)    
    # logger.info("end predict")
    # 计算预测方差
    variance = calculate_predicted_variance(model, X_test)
    
    return predictions, variance

def get_final_estimator(model):
    if hasattr(model, 'steps'):  # 检查是否是 Pipeline
        return model.steps[-1][1]  # 返回 Pipeline 的最后一个估计器
    return model

def calculate_predicted_variance(model, X_test):
    final_estimator = get_final_estimator(model)
    
    if hasattr(final_estimator, 'predict_proba'):
        proba = final_estimator.predict_proba(X_test)
        return np.var(proba, axis=1)
    elif hasattr(final_estimator, 'estimators_'):
        predictions = np.array([tree.predict(X_test.values) for tree in final_estimator.estimators_])
        return np.var(predictions, axis=0)
    else:
        logger.warning("模型不提供方差估计，返回固定值")
        return np.full(X_test.shape[0], 1e-6)


# 分配相等的权重
def equal_weight_portfolio(expected_returns):
    n = len(expected_returns)
    equal_weights = np.ones(n) / n
    return equal_weights

# 优化投资组合权重
def optimize_portfolio(expected_returns, cov_matrix, lambda_risk_aversion, max_iter=1000):
    n = len(expected_returns)
    
    expected_returns = np.array(expected_returns).flatten()


    def objective(weights):
        portfolio_return = np.sum(expected_returns * weights)
        portfolio_volatility = np.sqrt(np.dot(weights.T, np.dot(cov_matrix, weights)))
        return -(portfolio_return - lambda_risk_aversion * portfolio_volatility)

    constraints = (
        {'type': 'eq', 'fun': lambda x: np.sum(x) - 1},  # 权重之和为1
        # {'type': 'ineq', 'fun': lambda x: 0.5 - np.max(x)}  # 单个资产最大权重为50%
    )
    bounds = tuple((0, 1) for _ in range(n))  # 权重在0到1之间

    result = minimize(objective, n * [1./n], method='SLSQP', bounds=bounds, 
                      constraints=constraints, options={'maxiter': max_iter})
    
    if not result.success:
        logger.warning(f"Optimization failed: {result.message}")
        return np.ones(n) / n  # 返回等权重作为后备方案
    
    return result.x

def predict_returns(df, start_date, next_month, features, target, model):
    try:
        logger.info(f"Predicting returns from {start_date} to {next_month}")
        
        # Filter data for the training period (up to and including start_date)
        train = df[df.index.get_level_values('date') <= start_date]
        
        # Filter data for the test period (next_month only)
        test = df[df.index.get_level_values('date') == next_month]

        logger.info(f"Train data shape: {train.shape}")
        logger.info(f"Test data shape: {test.shape}")
        
        X_train, y_train = prepare_training_data(train, features, target)
        X_test = test[features]    
        logger.info(f"X_train shape: {X_train.shape}")
        logger.info(f"y_train shape: {y_train.shape}")
        logger.info(f"X_test shape: {X_test.shape}")
        
        if X_train.empty or y_train.empty:
            raise ValueError("Training data is empty")
        
        if X_test.empty:
            raise ValueError(f"No data available for the next month: {next_month}")
        
        predictions, predicted_variance = fit_and_predict(model, X_train, y_train, X_test)
        logger.info(f"Predictions shape: {predictions.shape}")
        logger.info(f"Predicted variance shape: {predicted_variance.shape}")
                
        # Get actual returns for the test period (next_month)
        actual_returns = test[target]

        # 添加这些日志语句
        logger.info(f"actual_returns shape: {actual_returns.shape}, type: {type(actual_returns)}")
        logger.info(f"predictions shape: {predictions.shape}, type: {type(predictions)}")
        
        # 确保 predictions 和 actual_returns 都是 DataFrame
        predictions_df = pd.DataFrame(predictions, index=actual_returns.index, columns=actual_returns.columns)
        # metrics = calculate_model_metrics(actual_returns, predictions_df)
        # logger.info(f"Model Evaluation Metrics: {metrics}")

        return predictions_df, pd.Series(predicted_variance, index=actual_returns.index), actual_returns
        
    except Exception as e:
        logger.error(f"Error in predict_returns: {str(e)}")
        raise


def calculate_model_metrics(y_true, y_pred):
    """
    计算模型评估指标：MSE, RMSE, MAE, R2
    
    参数:
    y_true (pd.DataFrame): 实际值
    y_pred (pd.DataFrame): 预测值
    
    返回:
    dict: 包含 mse, rmse, mae, r2 的字典
    """
    # 确保输入是DataFrame并且只有一列
    # assert isinstance(y_true, pd.DataFrame) and isinstance(y_pred, pd.DataFrame), "Inputs must be DataFrames"
    # assert y_true.shape[1] == 1 and y_pred.shape[1] == 1, "Inputs must have only one column"
    
    # 计算指标
    mse = mean_squared_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)
    
    return mse, rmse, mae, r2

def get_risk_free_rate(featured_data, date):
    rf_data = featured_data.loc[featured_data.index.get_level_values('date') == date, 'TB3MS']
    if len(rf_data) > 0:
        annual_rf_rate = rf_data.iloc[0] / 100
        logger.info(f"Annual Risk-free rate: {annual_rf_rate}")
        monthly_rf_rate = (1 + annual_rf_rate) ** (1/12) - 1
        logger.info(f"Monthly Risk-free rate: {monthly_rf_rate}")
    else:
        logger.warning(f"No risk-free rate data for {date}. Using average of last 3 months as default.")
        # 使用最近3个月的平均值作为默认值
        last_3_months = featured_data.loc[featured_data.index.get_level_values('date') < date, 'TB3MS'].tail(3)
        if len(last_3_months) > 0:
            annual_rf_rate = last_3_months.mean() / 100
            monthly_rf_rate = (1 + annual_rf_rate) ** (1/12) - 1
        else:
            logger.warning("No historical risk-free rate data available. Using 0 as default.")
            monthly_rf_rate = 0
    return monthly_rf_rate

In [694]:
def estimate_optimal_covariance(historical_data, current_tickers, target_column, min_history_length=252, min_data_pct=0):
    """
    估计最优协方差矩阵，处理不同时间点股票不同的问题，并减少填充的影响。
    
    参数:
    historical_data (pd.DataFrame): 包含历史数据的DataFrame，索引为MultiIndex（日期和股票代码）
    current_tickers (list): 当前时间点的股票代码列表
    target_column (str): 目标列名（例如，'returns'）
    min_history_length (int): 用于估计的最小历史数据长度
    min_data_pct (float): 股票需要的最小数据百分比
    
    返回:
    np.array: 估计的协方差矩阵
    """    
    historical_data = historical_data.sort_index()
    dates = historical_data.index.get_level_values('date').unique()
    recent_dates = dates[-min_history_length:] if len(dates) > min_history_length else dates
    
    recent_data = historical_data.loc[historical_data.index.get_level_values('date').isin(recent_dates)]
    recent_data = recent_data[recent_data.index.get_level_values('TICKER').isin(current_tickers)]
    
    if recent_data.empty:
        raise ValueError("No recent data available for the specified tickers")
    
    pivot_data = recent_data[target_column].unstack(level='TICKER')
    
    # 计算每个股票的数据完整性
    data_completeness = pivot_data.notna().sum() / len(pivot_data)
    valid_tickers = data_completeness[data_completeness >= min_data_pct].index.tolist()

    logger.info(f"estimate: valid tickers: {valid_tickers}")
    
    if not valid_tickers:
        raise ValueError("No stocks with sufficient data")
    
    valid_data = pivot_data[valid_tickers]

    # logger.info(f"estimate: valid_data: {valid_data.head()}")

    
    # 使用线性插值填充缺失值
    filled_data = valid_data.interpolate(method='linear', axis=0).fillna(method='ffill').fillna(method='bfill')
    
    # 计算每个股票的平均收益率和波动率
    mean_returns = filled_data.mean()
    std_returns = filled_data.std()
    
    # 对于缺失值较多的股票，使用其历史平均收益率和波动率来模拟数据
    for ticker in valid_tickers:
        missing_dates = filled_data[ticker].isna()
        if missing_dates.sum() > 0:
            simulated_returns = np.random.normal(mean_returns[ticker], std_returns[ticker], size=missing_dates.sum())
            filled_data.loc[missing_dates, ticker] = simulated_returns
    
    # 计算加权系数
    weights = data_completeness[valid_tickers].values
    weights = weights / weights.sum()
    
    # 加权数据
    weighted_data = filled_data * weights
    
    # 使用Ledoit-Wolf方法估计协方差矩阵
    lw = LedoitWolf()
    # logger.info("start lw fitting")
    cov_matrix = lw.fit(weighted_data.values).covariance_
    # logger.info("end lw fitting")
    
    if isinstance(pivot_data.columns, pd.MultiIndex):
        valid_tickers = [t[1] if isinstance(t, tuple) else t for t in valid_tickers]
        
    return cov_matrix, valid_tickers

In [695]:
# def process_single_period(df, featured_data, start_date, next_month, opt_features, target, model, lambda_risk_aversion):
#     try:
#         # logger.info(f"Processing period from {start_date} to {next_month}")
        
#         # historical_data = df[df.index.get_level_values('date') <= start_date]
#         # logger.info(f"Historical data shape: {historical_data.shape}")
        
#         # logger.info("About to call predict_returns function")

#         # predicted_returns, predicted_variance, actual_returns = predict_returns(df, start_date, next_month, opt_features, target, model)

#         # logger.info("predict_returns function completed")

#         # 获取下一个月的股票列表
#         next_month_stocks = df.loc[next_month].index.get_level_values('TICKER')
#         logger.info(f"Next month stocks: {next_month_stocks}")
#         logger.info(f"Number of stocks for next month: {len(next_month_stocks)}")

#         # # 确保 predicted_returns 和 actual_returns 具有相同的长度和顺序
#         # if len(predicted_returns) != len(next_month_stocks):
#         #     raise ValueError(f"Mismatch between predicted returns length ({len(predicted_returns)}) and number of stocks ({len(next_month_stocks)})")
        
#         # # # 使用这些股票的历史收益率数据计算协方差矩阵
#         # # historical_returns = historical_data.loc[(slice(None), next_month_stocks), target].unstack().pct_change().dropna()
#         # # cov_matrix = historical_returns.cov()
        
#         # # logger.info(f"Covariance matrix shape: {cov_matrix.shape}")
#         # # logger.info(f"Predicted returns shape: {predicted_returns.shape}")
        
#         # # if cov_matrix.shape[0] != len(predicted_returns):
#         # #     raise ValueError(f"Mismatch between covariance matrix shape {cov_matrix.shape} and predicted returns length {len(predicted_returns)}")


#         # # optimal_weights = optimize_portfolio(predicted_returns, cov_matrix, lambda_risk_aversion)
        
#         # # 在 process_single_period 函数中
#         # current_tickers = df.loc[next_month].index.get_level_values('TICKER')
#         # cov_matrix = estimate_optimal_covariance(historical_data, current_tickers, target)

#         # # 确保 predicted_returns 和 cov_matrix 的顺序一致
#         # predicted_returns_aligned = predicted_returns[np.isin(current_tickers, df.loc[next_month].index)]
#         # cov_matrix_aligned = cov_matrix[np.isin(current_tickers, df.loc[next_month].index)][:, np.isin(current_tickers, df.loc[next_month].index)]

#         logger.info(f"Processing period from {start_date} to {next_month}")
        
#         historical_data = df[df.index.get_level_values('date') <= start_date]
#         logger.info(f"Historical data shape: {historical_data.shape}")
        
#         # logger.info("About to call predict_returns function")

#          # 估计协方差矩阵
#         cov_matrix, valid_tickers = estimate_optimal_covariance(historical_data, current_tickers, target)
        
#         predicted_returns, predicted_variance, actual_returns = predict_returns.loc[valid_tickers](df, start_date, next_month, opt_features, target, model)

#         # logger.info("predict_returns function completed")

#         # 获取下一个月的股票列表
#         current_tickers = df.loc[next_month].index.get_level_values('TICKER').tolist()
#         # logger.info(f"Next month stocks: {current_tickers}")
#         logger.info(f"Number of stocks for next month: {len(current_tickers)}")

       
#         logger.info(f"Covariance matrix shape: {cov_matrix.shape}")
#         logger.info(f"Predicted returns shape: {predicted_returns.shape}")
        
#         if cov_matrix.shape[0] != len(predicted_returns):
#             raise ValueError(f"Mismatch between covariance matrix shape {cov_matrix.shape} and predicted returns length {len(predicted_returns)}")

#         optimal_weights = optimize_portfolio(predicted_returns, cov_matrix, lambda_risk_aversion)
        
#         # optimal_weights = optimize_portfolio(predicted_returns_aligned, cov_matrix_aligned, lambda_risk_aversion)

#         if optimal_weights is None:
#             logger.warning("Using equal weights due to optimization failure")
#             optimal_weights = np.ones(len(predicted_returns)) / len(predicted_returns)
        
#         logger.info(f"Optimal weights shape: {optimal_weights.shape}")
        
#         portfolio_composition = display_portfolio_composition(optimal_weights, next_month_stocks)
#         print(f"Portfolio Composition for {next_month.date()}:")
#         print(portfolio_composition)
        
#         monthly_rf_rate = get_risk_free_rate(featured_data, next_month)
        
#         monthly_portfolio_returns = np.sum(optimal_weights * actual_returns.values)
        
#         portfolio_return = monthly_portfolio_returns
#         logger.info(f"Monthly Portfolio return: {portfolio_return}")
        
#         # 使用预测收益的方差来计算投资组合波动率
#         portfolio_variance = np.sum((optimal_weights ** 2) * predicted_variance)
#         portfolio_volatility = np.sqrt(portfolio_variance)
#         logger.info(f"Monthly Portfolio volatility: {portfolio_volatility}")
        
#         if portfolio_volatility > 0:
#             monthly_sharpe_ratio = (portfolio_return - monthly_rf_rate) / portfolio_volatility
#             annualized_sharpe_ratio = monthly_sharpe_ratio * np.sqrt(12)
#         else:
#             monthly_sharpe_ratio = np.nan
#             annualized_sharpe_ratio = np.nan
        
#         logger.info(f"Monthly Sharpe ratio: {monthly_sharpe_ratio}")
#         logger.info(f"Annualized Sharpe ratio: {annualized_sharpe_ratio}")
        
#         return portfolio_return, monthly_sharpe_ratio, mse, rmse, mae, r2
#     except Exception as e:
#         logger.error(f"Error processing period {start_date} to {next_month}: {str(e)}")
#         raise
        
# # 使用示例：
# date = pd.Timestamp('2017-12-31')
# next_month = date + pd.offsets.MonthEnd(1)
# lambda_risk_aversion = 0.5
# portfolio_return, sharpe_ratio, mse, rmse, mae, r2 = process_single_period(opt_data, featured_data, date, next_month, opt_features, target, pipeline, lambda_risk_aversion)
# if portfolio_return is not None and sharpe_ratio is not None:
#     print(f"Portfolio Return: {portfolio_return:.4f}")
#     print(f"Sharpe Ratio: {sharpe_ratio:.4f}")
#     print(f"MSE: {mse:.4f}")
#     print(f"RMSE: {rmse:.4f}")
#     print(f"MAE: {mae:.4f}")
#     print(f"R²: {r2:.4f}")


In [716]:
import numpy as np
import pandas as pd
from sklearn.covariance import LedoitWolf
from sklearn.metrics import mean_squared_error, mean_absolute_error

# ... [其他辅助函数保持不变] ...

def process_single_period(df, featured_data, start_date, next_month, opt_features, target, model, lambda_risk_aversion):
    try:
        logger.info(f"Processing period from {start_date} to {next_month}")
        
        historical_data = df[df.index.get_level_values('date') <= start_date]
        logger.info(f"Historical data shape: {historical_data.shape}")
        
        # predict
        predicted_returns, predicted_variance, actual_returns = predict_returns(df, start_date, next_month, opt_features, target, model)

        logger.info(f"Predicted returns shape: {predicted_returns.shape}")
        logger.info(f"Predicted variance shape: {predicted_variance.shape}")
        logger.info(f"Actual returns shape: {actual_returns.shape}")
        
        if actual_returns.empty:
            raise ValueError("No valid actual returns data after filtering")
        
        # get ticker
        current_tickers = df.loc[next_month].index.get_level_values('TICKER').tolist()
        logger.info(f"Number of stocks for next month: {len(current_tickers)}")

        # get cov based on current ticker's return
        # logger.info(f"process_: current tickers: {current_tickers}")
        cov_matrix, valid_tickers = estimate_optimal_covariance(historical_data, current_tickers, target)
        # logger.info(f"process_: valid tickers: {valid_tickers}")
        # logger.info(f"process_: cov_matrix: {cov_matrix}")


        logger.info(f"Covariance matrix shape: {cov_matrix.shape}")
        logger.info(f"Number of valid tickers: {len(valid_tickers)}")
        
        # 确保 predicted_returns 和 cov_matrix 使用相同的股票
        valid_mask = np.isin(current_tickers, valid_tickers)
        predicted_returns_valid = predicted_returns[valid_mask]
        predicted_variance_valid = predicted_variance[valid_mask]
        
        # 处理 actual_returns_valid 可能是 DataFrame 的情况
        if isinstance(actual_returns, pd.DataFrame):
            logger.info(f"actual_returns is a DataFrame with shape: {actual_returns.shape}")
            actual_returns_valid = actual_returns.loc[actual_returns.index.get_level_values('TICKER').isin(valid_tickers)]
            logger.info(f"actual_returns_valid after filtering: {actual_returns_valid.shape}")
            if actual_returns_valid.empty:
                raise ValueError("No valid actual_returns_valid data after filtering")
            actual_returns_valid = actual_returns_valid.values.flatten()
        else:
            logger.info(f"actual_returns is a numpy array with shape: {actual_returns.shape}")
            actual_returns_valid = actual_returns[valid_mask]
        
        valid_tickers = np.array(current_tickers)[valid_mask]
        # logger.info(f"valid_tickers: {valid_tickers}")

        logger.info(f"Shape of predicted_returns_valid: {predicted_returns_valid.shape}")
        logger.info(f"Shape of actual_returns_valid: {actual_returns_valid.shape}")
        logger.info(f"Number of valid tickers: {len(valid_tickers)}")

        if len(predicted_returns_valid) == 0 or len(actual_returns_valid) == 0:
            logger.warning("No valid data for this period. Skipping...")
            return None

        if cov_matrix.shape[0] != len(predicted_returns_valid):
            raise ValueError(f"Mismatch between covariance matrix shape {cov_matrix.shape} and predicted returns length {len(predicted_returns_valid)}")

        optimal_weights = optimize_portfolio(predicted_returns_valid, cov_matrix, lambda_risk_aversion)
        
        logger.info(f"Shape of optimal_weights: {optimal_weights.shape}")
        # logger.info(f"optimal_weights: {optimal_weights}")

        # 显示投资组合构成
        portfolio_composition, portfolio_str = display_portfolio_composition(optimal_weights, valid_tickers)
        logger.info(f"Portfolio Composition for {next_month}:")
        logger.info(portfolio_str)

        portfolio_return = np.dot(optimal_weights , predicted_returns_valid).item()
        logger.info(f"Calculated portfolio return: {portfolio_return}")

        # portfolio_volatility = np.sqrt(np.dot(optimal_weights.T, np.dot(cov_matrix, optimal_weights)))
        portfolio_volatility = np.sqrt(np.dot(optimal_weights, predicted_variance_valid).item())
        logger.info(f"portfolio_volatility: {portfolio_volatility}")

        rf_rate = get_risk_free_rate(featured_data, next_month)
        
        sharpe_ratio = (portfolio_return - rf_rate) / portfolio_volatility #if portfolio_volatility > 0 else np.nan
        logger.info(f"sharpe_ratio: {sharpe_ratio}")

        mse, rmse, mae, r2 = calculate_model_metrics(actual_returns_valid, predicted_returns_valid)
        
        # print(portfolio_composition.to_string(index=False))

        return portfolio_return, sharpe_ratio, mse, rmse, mae, r2

    except Exception as e:
        logger.error(f"Error processing period {start_date} to {next_month}: {str(e)}")
        logger.error(f"Error details: {str(e)}", exc_info=True)
        return None

# 使用示例
date = pd.Timestamp('2017-12-31')
next_month = date + pd.offsets.MonthEnd(1)
lambda_risk_aversion = 0.9

result = process_single_period(opt_data, featured_data, date, next_month, opt_features, target, pipeline, lambda_risk_aversion)

if result is not None:
    portfolio_return, sharpe_ratio, mse, rmse, mae, r2 = result
    print(f"Portfolio Return: {portfolio_return:.4f}")
    print(f"Sharpe Ratio: {sharpe_ratio:.4f}")
    print(f"MSE: {mse:.4f}")
    print(f"RMSE: {rmse:.4f}")
    print(f"MAE: {mae:.4f}")
    print(f"R²: {r2:.4f}")
    print("\nPortfolio Composition:")
else:
    print("No valid data for this period.")

2024-08-13 20:52:56,805 - INFO - Processing period from 2017-12-31 00:00:00 to 2018-01-31 00:00:00
2024-08-13 20:52:56,810 - INFO - Historical data shape: (1124, 40)
2024-08-13 20:52:56,811 - INFO - Predicting returns from 2017-12-31 00:00:00 to 2018-01-31 00:00:00
2024-08-13 20:52:56,812 - INFO - Train data shape: (1124, 40)
2024-08-13 20:52:56,813 - INFO - Test data shape: (27, 40)
2024-08-13 20:52:56,814 - INFO - X_train shape: (1124, 39)
2024-08-13 20:52:56,815 - INFO - y_train shape: (1124, 1)
2024-08-13 20:52:56,815 - INFO - X_test shape: (27, 39)
2024-08-13 20:52:58,923 - INFO - Predictions shape: (27,)
2024-08-13 20:52:58,924 - INFO - Predicted variance shape: (27,)
2024-08-13 20:52:58,924 - INFO - actual_returns shape: (27, 1), type: <class 'pandas.core.frame.DataFrame'>
2024-08-13 20:52:58,925 - INFO - predictions shape: (27,), type: <class 'numpy.ndarray'>
2024-08-13 20:52:58,925 - INFO - Predicted returns shape: (27, 1)
2024-08-13 20:52:58,926 - INFO - Predicted variance sh

Portfolio Return: 0.0362
Sharpe Ratio: 0.9496
MSE: 0.0004
RMSE: 0.0188
MAE: 0.0152
R²: -1.1461

Portfolio Composition:


In [697]:
def rolling_portfolio_optimization(opt_data, featured_data, start_date, end_date, features, target, model, lambda_risk_aversion, window_size):
    dates = pd.date_range(start_date, end_date, freq='M')
    portfolio_returns = []
    sharpe_ratios = []
    mses = []
    rmses = []
    maes = []
    r2s = []

    for i in range(len(dates) - window_size):
        train_start = dates[i]
        train_end = dates[i + window_size - 1]
        next_month = dates[i + window_size]
        
        logger.info(f"Training from {train_start} to {train_end}, predicting for {next_month}")
        
        # 训练模型
        train_data = get_data_for_date_range(opt_data, train_start, train_end)
        X_train, y_train = prepare_training_data(train_data, features, target)
        model.fit(X_train, y_train)
        
        # 处理单个时期并计算投资组合回报率和夏普比率
        portfolio_return, sharpe_ratio, mse, rmse, mae, r2 = process_single_period(opt_data, featured_data, train_end, next_month, features, target, model, lambda_risk_aversion)
        portfolio_returns.append(portfolio_return)
        sharpe_ratios.append(sharpe_ratio)
        mses.append(mse)
        rmses.append(rmse)
        maes.append(mae)
        r2s.append(r2)
        
    
    return portfolio_returns, sharpe_ratios, mses, rmses, maes, r2s



In [717]:
# 使用示例：
start_date = pd.Timestamp('2015-10-31')
end_date = pd.Timestamp('2016-12-31')
# start_date = opt_data.index.get_level_values('date').min()
# end_date = opt_data.index.get_level_values('date').max()

lambda_risk_aversion = 0.9
window_size = 12  # 每个持有期为12个月

portfolio_returns, sharpe_ratios, mses, rmses, maes, r2s = rolling_portfolio_optimization(opt_data, featured_data, start_date, end_date, opt_features, target, pipeline, lambda_risk_aversion, window_size)


# 计算平均夏普比率
average_sharpe_ratio = np.mean(sharpe_ratios)
print(f"Average Sharpe Ratio: {average_sharpe_ratio}")

print(f"Average MSE: {np.mean(mses)}")
print(f"Average RMSE: {np.mean(rmses)}")
print(f"Average MAE: {np.mean(maes)}")
print(f"Average R2: {np.mean(r2s)}")


/var/folders/pp/0c_n54bd7wd4nrd15cp7l12m0000gn/T/ipykernel_23571/1559073234.py:2: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  dates = pd.date_range(start_date, end_date, freq='M')
2024-08-13 20:53:10,654 - INFO - Training from 2015-10-31 00:00:00 to 2016-09-30 00:00:00, predicting for 2016-10-31 00:00:00
2024-08-13 20:53:11,070 - INFO - Processing period from 2016-09-30 00:00:00 to 2016-10-31 00:00:00
2024-08-13 20:53:11,071 - INFO - Historical data shape: (713, 40)
2024-08-13 20:53:11,071 - INFO - Predicting returns from 2016-09-30 00:00:00 to 2016-10-31 00:00:00
2024-08-13 20:53:11,072 - INFO - Train data shape: (713, 40)
2024-08-13 20:53:11,073 - INFO - Test data shape: (27, 40)
2024-08-13 20:53:11,074 - INFO - X_train shape: (713, 39)
2024-08-13 20:53:11,074 - INFO - y_train shape: (713, 1)
2024-08-13 20:53:11,074 - INFO - X_test shape: (27, 39)
2024-08-13 20:53:12,073 - INFO - Predictions shape: (27,)
2024-08-13 20:53:12,074

Average Sharpe Ratio: 0.6487291670022334
Average MSE: 0.0003617884602123877
Average RMSE: 0.018491753016163157
Average MAE: 0.0141453012345679
Average R2: -1.0021433702474345
